## Call the dockerized local model

Run before:

```make docker-build```

```make docker-run```

Once the container is running locally, you can call the API:


In [32]:
import requests
PORT = 80
LOCAL_URL = f"http://localhost:{PORT}"

response = requests.get(
    LOCAL_URL,
)

# Test connectivity
response.text

'{"message":"pong"}'

In [34]:
response = requests.get(
    f"{LOCAL_URL}/predict",
    params={"prompt": "What is the capital of France?"}
)

response.text

'{"detail":"Not Found"}'

## Call the API for the GCP deployed model

In [4]:
import google.auth
import google.auth.transport.requests
import google.oauth2.id_token
import requests

def get_gcp_bearer_token(ENDPOINT_URL: str = "https://detect-prompt-injection-941427557247.us-central1.run.app"):
    """
    Get the bearer token for the GCP deployed model

    ENDPOINT_URL: str The endpoint URL of the GCP deployed model.

    `GOOGLE_APPLICATION_CREDENTIALS` is set in the environment to the keys json file
    """
    credentials, project = google.auth.default(scopes=['https://www.googleapis.com/auth/cloud-platform'])
    auth_req = google.auth.transport.requests.Request()
    return google.oauth2.id_token.fetch_id_token(auth_req, ENDPOINT_URL)

def predict(prompt: str, ENDPOINT_URL: str = "https://detect-prompt-injection-941427557247.us-central1.run.app") -> dict[str, str | float]:
    """
    Predict the prompt injection

    prompt: str The prompt to predict
    ENDPOINT_URL: str The endpoint URL of the GCP deployed model.
    """

    headers = {
        'Authorization': f'Bearer {get_gcp_bearer_token()}',
        'Content-Type': 'application/json'
    }

    response = requests.get(
        f"{ENDPOINT_URL}/predict",
        headers=headers,
        params={"prompt": prompt}
    )

    return response.json()
    
response = predict("What is the capital of France?")
response


{'label': 'benign', 'probability': 0.9996808767318726}